<a href="https://colab.research.google.com/github/a-brice/projet/blob/main/ml-minmax-algo/AI_tictactoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Visualisation de l'algorithme du MinMax par le jeu du morpion**

In [37]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  4 01:04:22 2021

@author: brice
"""


import pandas as pd
import numpy as np
import time
   

In [38]:

def creation_table():
    plateau = np.array([['*' for i in range(3)] for j in range(3)])
    return plateau


    
def afficher_plateau(plateau):
    df = pd.DataFrame(plateau, index=['','',''], columns=['','',''])
    print('\n', df, '\n', sep='')
      

Définitons des méthodes nécessaire concernant l'état de la partie jouée.

In [39]:
  
def actions(table):
    ''' Retourne une liste d'action (placement point) pouvant être jouer '''
    listepossible = []
    for i in range(3):
        for j in range(3):
            if table[i,j] == '*':
                listepossible.append((i,j))
    return listepossible


def terminal_test(table):
    ''' Détermine si l'état du plateau est un état final '''
    if not actions(table):
        return True
    
    return True if IA('X').utility(table) != 0 or IA('O').utility(table) != 0 else False


def results(table,action, symbol):
    ''' Permet de jouer de faire une action sur le plateau ''' 
    tab = np.array(table)

    if not action in [(i,j) for i in range(3) for j in range(3)]:
        print('Placement hors du plateau')
    elif not action in actions(table):
        print('Impossible : il ya déjà un pion')
    else:
        tab[action] = symbol
        
    return tab
        

# Définition de la classe IA 
Cette classe permettra de jouer contre l'ordinateur qui sera d'une intélligence et pourras prédire à l'avance les coup pouvant être jouer afin de les contrer au mieux

In [40]:
               
class IA:
    
    def __init__(self,symbol):
        self.symbol = symbol
        self.symbol_inv = 'X' if symbol == 'O' else 'O'
            
    
        
    def utility(self, table):
        u = 0
        
        # Diagonale 1
        if [table[i,i] for i in range(3)].count(self.symbol) == 3:
            u = 1
        elif [table[i,i] for i in range(3)].count(self.symbol_inv) == 3:
            u = -1
        #Diagonale 2
        elif [table[i] for i in zip(range(3),range(2,-1,-1))].count(self.symbol) == 3:
            u = 1
        elif [table[i] for i in zip(range(3),range(2,-1,-1))].count(self.symbol_inv) == 3:
            u = -1
        else:   # Ligne et Colonne
          for i in range(3):
            if list(table[:,i]).count(self.symbol) >= 3 or list(table[i,:]).count(self.symbol) >= 3:
              u = 1
            if list(table[:,i]).count(self.symbol_inv) >= 3 or list(table[i,:]).count(self.symbol_inv) >= 3:
              u = -1
         
        return u
    
    
    
    def minimax_decision(self, table):
        
        res = {}

        for a in actions(table):
            res[tuple(a)] = self.min_value(results(table,a,self.symbol)) 
   
        return max(res,key=res.get)
        
            
        
    def min_value(self, table):
        if terminal_test(table):
            return self.utility(table)
    
        result = [results(table,a,self.symbol_inv) for a in actions(table)]
        v = min(self.max_value(res) for res in result)
        return v
    
    
    
    def max_value(self, table):
        if terminal_test(table):
            return self.utility(table)
    
        result = [results(table,a, self.symbol) for a in actions(table)]
        v = max(self.min_value(res) for res in result)
        v = self.utility(table)
        return v



# Création d'un mode **SOLO**
Cette méthode permettra au joueur d'affronter l'ordinateur 

In [41]:

def mode_solo():
    global table
    ia = IA('O')
    
    while not terminal_test(table):
        saisi = tuple()
        while not saisi:
            saisi = tuple(input('Indiquer la position à jouer : '))
            saisi = tuple([int(s) for s in saisi if s.isnumeric()])
            print(saisi, end='')
            if not saisi in tuple((i,j)for i in range(3) for j in range(3)):
                print(' -> La position indiquée est incohérente')
                saisi = tuple()
            elif table[saisi] != '*':
                print(" -> Un pion a déjà été placée ")
                saisi = tuple()
            
        table = results(table, saisi,'X')
        afficher_plateau(table)
        
        if terminal_test(table):  #Juste après la saisie joueur
            break
        
        print("\n","L'IA joue : ", sep='')
        time.sleep(1)
        actIA = ia.minimax_decision(table)
        table = results(table,actIA, ia.symbol)
        afficher_plateau(table)
        
    if ia.utility(table) == 1:
        print("C'est normal, tu ne pouvais pas rivaliser contre moi")
    elif ia.utility(table) == 0:
        print('Match nul.. On relance !')
    elif ia.utility(table) == -1:
        print('Coup de chance pour toi ...')
    


# Création d'un mode **DUEL**
La méthode suivante permettra d'observer une partie opposant 2 **IA** dotées du même niveau d'intélligence

In [42]:

def duel_IA():

    global table
    ia1 = IA('O')
    ia2 = IA('X')
    
    premiercoup = tuple([np.random.randint(0,3),np.random.randint(0,3)])
    table = results(table,premiercoup, ia2.symbol)
    print("Le premier coup (aléatoire) a été joué par l'IA2")
    afficher_plateau(table)
    time.sleep(3)
    
    while not terminal_test(table):
        print("\n","L'IA1 joue : ", sep='')
        time.sleep(3)
        actIA1 = ia1.minimax_decision(table)
        table = results(table,actIA1, ia1.symbol)
        afficher_plateau(table)
        
        if terminal_test(table):# après le coup du 1er IA
            break
        
        print("\n","L'IA2 joue : ", sep='')
        time.sleep(3)
        actIA2 = ia2.minimax_decision(table)
        table = results(table,actIA2, ia2.symbol)
        afficher_plateau(table)
    
    
    if ia1.utility(table) == 1:
        print("vainqueur : IA1")
    elif ia1.utility(table) == 0:
        print('Match nul royale')
    elif ia1.utility(table) == -1:
        print('Vainqueur : IA2')


# **Programme principal**

In [43]:

if __name__ == '__main__':
    table = creation_table()
    afficher_plateau(table)
    choice = int(input('1 : You vs IA\t||\t2: IA vs IA \n\nVotre choix : '))
    if choice == 1:
        mode_solo()
    elif choice == 2:
        duel_IA()
        


         
  *  *  *
  *  *  *
  *  *  *

1 : You vs IA	||	2: IA vs IA 

Votre choix : 1
Indiquer la position à jouer : 1,1
(1, 1)
         
  *  *  *
  *  X  *
  *  *  *


L'IA joue : 

         
  O  *  *
  *  X  *
  *  *  *

Indiquer la position à jouer : 2,0
(2, 0)
         
  O  *  *
  *  X  *
  X  *  *


L'IA joue : 

         
  O  *  O
  *  X  *
  X  *  *

Indiquer la position à jouer : (0,1)
(0, 1)
         
  O  X  O
  *  X  *
  X  *  *


L'IA joue : 

         
  O  X  O
  *  X  *
  X  O  *

Indiquer la position à jouer : 2,2
(2, 2)
         
  O  X  O
  *  X  *
  X  O  X


L'IA joue : 

         
  O  X  O
  O  X  *
  X  O  X

Indiquer la position à jouer : 2,1
(2, 1) -> Un pion a déjà été placée 
Indiquer la position à jouer : 1,2
(1, 2)
         
  O  X  O
  O  X  X
  X  O  X

Match nul.. On relance !
